# Kalman Filtering and Recurrent Neural Networks for Fuel Moisture

In [ ]:
# Environment
import numpy as np
import pandas as pd
from MesoPy import Meso
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import pytz
# Local modules for handling data and running moisture models
import data_funcs as datf
import moisture_models as mod

meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

### RAWS Data

In [ ]:
vars='air_temp,relative_humidity,precip_accum,fuel_moisture'
time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM 

meso_ts = m.timeseries(time_start, time_end, 
                       stid="BKCU1", vars=vars)
station = meso_ts['STATION'][0]

In [ ]:
raws_dat = datf.format_raws(station)

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(raws_dat['fm'],linestyle='-',c='k',label='10-h fuel data')
plt.title(station['STID'] + ' 10 h fuel moisture data')
plt.xlabel('Time (hours)') 
plt.ylabel('Fuel moisture content (%)')
plt.legend()

### RTMA Data

In [ ]:
import subprocess,os
def load_rtma(path,file,reload=0):
    url='http://math.ucdenver.edu/~jmandel/rtma/' + path 
    if os.path.exists(file):
      if reload:
        print(file + ' already exists, removing')
        os.remove(file)
      else:
        print(file + ' already exists, exiting')
        # add checking size here
        return 0
    try:
      ret = subprocess.check_output(['wget','--no-clobber','--output-document='+ file, url,],stderr=subprocess.STDOUT).decode() # execute command from python strings
      if os.path.exists(file):
        print('loaded ' + url + ' as ' + file)
        return 0
      else: 
        print('file transfer completed, but the file is missing? ' + url)  
      return 1
    except:
      print('file transfer failed: ' + url)
      return 2

In [ ]:
def rtma_grib(t,var):
    tpath = '%4i%02i%02i/%02i' % (t.year, t.month, t.day, t.hour)  # remote path on server
    tstr  = '%4i%02i%02i%02i_' % (t.year, t.month, t.day, t.hour)  # time string for local path
    gribfile = os.path.join('data',tstr + var + '.grib')
    remote = tpath + '/' + var + '.grib'
    if load_rtma(remote,gribfile):
        print('cannot load remote file',remote,'as',gribfile)
        return []
    else:
        try:
            gf=GribFile(gribfile)
            v = np.array(gf[1].values())
        except:
            print('cannot read grib file',gribfile)
            return []
        print('loaded ',gribfile,' containing array shape ',v.shape)
        return gf[1]   # grib message


In [ ]:
times = pd.date_range(start=time_start,end=time_end,freq='1H')
varnames=['temp','td','precipa']
j =    read_interp_rtma(varnames,times,obs_lat,obs_lon)      # temperature
for varname in varnames:
    j[varname]=j[varname].tolist() 
j['obs_lat']=obs_lat
j['obs_lon']=obs_lon
json.dump(j,open('rtma.json','w'),indent=4)
print('done')